In [2]:
%cd /content/drive/MyDrive/#temp/ai_pu_yun
!pip install -r requirements.txt
#!unzip arrays.zip

/content/drive/MyDrive/experience_job_application/companies-interviews/ai_pu_yun


In [11]:
# module import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os, sys, glob
import PIL
#from imutils.video import FPS
from queue import Queue
from datetime import datetime

# sklearn packages
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# tf packages
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import tensorflow_addons as tfa
# tensorboard initializations
"""
%load_ext tensorboard
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=0)
"""
# test gpu 
print("CUDA is running.") if tf.test.is_gpu_available() else print("CUDA is not running.")
#"""

from vit2 import VisionTransformer


CUDA is not running.


In [4]:
x_train_file, y_train_file = open("arrays/x_train.npy","br"), open("arrays/y_train.npy","br")
x_val_file, y_val_file = open("arrays/x_test.npy","br"), open("arrays/y_test.npy","br")
x_train, y_train = np.load(x_train_file), np.load(y_train_file)
x_val, y_val = np.load(x_val_file), np.load(y_val_file)

from sklearn.preprocessing import LabelEncoder
y_train = to_categorical(LabelEncoder().fit_transform(y_train))
y_val = to_categorical(LabelEncoder().fit_transform(y_val))


In [5]:
x_train.shape

(9882, 256, 256, 3)

In [6]:
"""
RUN THIS BEFORE TRAINING STARTS TO UPDATE MODEL CODENAME
"""
timenow = datetime.now().strftime("%m%d %H:%M:%S").split(' ')
datecode,timecode = timenow[0], ''
for i, s in enumerate(timenow[1].split(':')): 
    if i == 0: s = str(int(s)+8) #taiwan time
    timecode += s
code = datecode + '_' + timecode

print("This session's code:", code)

This session's code: 1109_193133


In [15]:
tfa.register_all()

In [ ]:

#strategy = tf.distribute.MirroredStrategy()
IMAGE_SIZE=256
PATCH_SIZE=7
LAYERS=8
D_MODEL=64
NUM_HEADS=4
MLP_DIM=128
LR=1e-3
WEIGHT_DECAY=5e-3
EPOCHS=20 
STEPS_PER_EPOCH=180

#with strategy.scope():
def create_model():
    model = VisionTransformer(
        image_size=IMAGE_SIZE,
        patch_size=PATCH_SIZE,
        num_layers=LAYERS,
        num_classes=2,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        mlp_dim=MLP_DIM,
        channels=3,
        dropout=0.1,
    )

    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.optimizers.Adam(learning_rate=LR),
        metrics=["accuracy"],
    )
    return model
    
model = create_model()
early_stop = tf.keras.callbacks.EarlyStopping(patience=10),
mcp = tf.keras.callbacks.ModelCheckpoint(filepath='weights/best.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='auto',
min_delta=0.0001, cooldown=0, min_lr=0)    

model.fit(
    x_train, y_train,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(x_val, y_val),
    callbacks=[early_stop, mcp, reduce_lr] #, tensorboard_callback],
)



In [ ]:
plt.subplot(121)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
create_folder = lambda path : os.mkdir(path) if not os.path.exists(path) else None
create_folder('models')
create_folder('weights')

MODEL_NAME = f'vit_{code}'
print(f"Saving to models/: : {MODEL_NAME}.h5")
#create_folder(f'models/{MODEL_NAME}')

# saving checkpoints
create_folder(f'weights/{code}')
model.save_weights(os.path.join('weights', f"{code}/{MODEL_NAME}"))

# saving model
model.save(f'models/vit_{code}',save_format='tf')

In [ ]:
with open('.vit_model_code.txt','w+') as f:
    f.write(f'models/vit_{code}.tf')
with open('.vit_weights_code.txt','w+') as f:
    f.write(f'weights/{code}/{MODEL_NAME}')


In [ ]:
model.summary()

Model: "vision_transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  9472      
_________________________________________________________________
transformer_block (Transform multiple                  33472     
_________________________________________________________________
transformer_block_1 (Transfo multiple                  33472     
_________________________________________________________________
transformer_block_2 (Transfo multiple                  33472     
_________________________________________________________________
transformer_block_3 (Transfo multiple                  33472     
_________________________________________________________________
transformer_block_4 (Transfo multiple           